# Optimizer Tests

This notebook implements static environments to test optimizers and ensure that they are correctly implemented and learn a fixed function, when mechanisms do not change over time and do not depend on other players.

In [ ]:
import os
import sys
root_path = os.path.abspath(os.path.join('..'))
if root_path not in sys.path:
    sys.path.append(root_path)

In [ ]:
import warnings
import pytest
import torch
import torch.nn as nn
from bnelearn.strategy import NeuralNetStrategy
from bnelearn.mechanism import StaticMechanism, StaticFunctionMechanism
from bnelearn.bidder import Bidder
from bnelearn.learner import ESPGLearner, PGLearner, AESPGLearner
from bnelearn.environment import AuctionEnvironment

In [ ]:
from torch.utils.tensorboard import SummaryWriter
import numpy as np
import matplotlib.pyplot as plt

# set up matplotlib
is_ipython = 'inline' in plt.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

In [ ]:
# Shared objects
cuda = torch.cuda.is_available()
device = 'cuda' if cuda else 'cpu'

hidden_nodes = [128,128,128]
input_length = 1
hidden_activations = [nn.SELU(), nn.SELU(), nn.SELU()]
dropout = 0.05

u_lo = 0
u_hi = 10

def strat_to_bidder(strategy, batch_size, player_position=None):
    return Bidder.uniform(u_lo,u_hi, strategy, batch_size = batch_size, player_position=player_position)

mechanism = StaticMechanism(cuda=cuda)
#mechanism = StaticFunctionMechanism(cuda=cuda)

In [ ]:
cuda

In [ ]:
run_dir = 'optimizer_tests'
run_name = '16'

BATCH_SIZE = 2**12 # 01: 2**18
epoch = 20000
print_epoch = 100
plot_epoch = 500

learner_hyperparams = {
    'population_size': 10,
    'sigma': 1.,
    'scale_sigma_by_model_size': True
}

learner3_hyperparams = {
    'population_size': 2,
    'sigma': 1.,
    'scale_sigma_by_model_size': False
}

optimizer_type = torch.optim.Adam#SGD#Adam
optimizer_hyperparams ={    
    #'lr': 1e-3,
    #'momentum': 0.5
}


#sigma = .1 #ES noise parameter #std 0.1
#n_perturbations = 32


logdir = os.path.join(root_path, 'notebooks', run_dir , run_name)

In [ ]:
def log_hyperparams(writer, e):
    pass
#     writer.add_scalar('hyperparams/batch_size', BATCH_SIZE, e)
#     writer.add_scalar('hyperparams/size_hidden_layer', SIZE_HIDDEN_LAYER, 0)
#     writer.add_scalar('hyperparams/learning_rate', LEARNING_RATE, e)
#     writer.add_scalar('hyperparams/momentum', MOMENTUM, e)
#     writer.add_scalar('hyperparams/sigma', sigma, e)
#     writer.add_scalar('hyperparams/n_perturbations', n_perturbations, e)

def optimal_bid(v):
    "truthful is optimal in this environment"
    return v

def plot_bid_function(fig, v,b, writer=None, e=None, plot_points=100):
    
    # subsample points and plot
    v = v.detach().cpu().numpy()[:plot_points]
    b= b.detach().cpu().numpy()[:plot_points]
    
    optimal = optimal_bid(v)
    fig = plt.gcf()
    plt.cla()
    plt.plot(v,b, 'o', v, optimal, 'r-')
    if is_ipython:
        #display.clear_output(wait=True)
        display.display(plt.gcf())
    if writer:
        writer.add_figure('eval/bid_function', fig, e)  

In [ ]:
model = NeuralNetStrategy(input_length,
                          hidden_nodes = hidden_nodes,
                          hidden_activations = hidden_activations, ensure_positive_output=torch.tensor([1.0]),
                          dropout = dropout).to(device)
bidder = strat_to_bidder(model, BATCH_SIZE, player_position=0)
 #bidder = strat_to_bidder(model, BATCH_SIZE)
#model.pretrain(bidder.valuations, 10)
env = AuctionEnvironment(
    mechanism,
    agents = [bidder],
    strategy_to_player_closure=strat_to_bidder,
    #max_env_size=1,
    batch_size = BATCH_SIZE,
    n_players=1
    )

learner = ESPGLearner(
    model, env, learner_hyperparams, optimizer_type, optimizer_hyperparams)

learner2 = PGLearner(model, env, {}, optimizer_type, optimizer_hyperparams)

learner3 = AESPGLearner(model, env, learner3_hyperparams, optimizer_type, optimizer_hyperparams)

print('n_params: ' + str(sum([p.numel() for p in model.parameters()])))
print(model)

In [ ]:
torch.cuda.empty_cache()

with SummaryWriter(logdir, flush_secs=30) as writer:
    log_hyperparams(writer, 0)
    fig = plt.figure()

    for e in range(epoch+1):
        #update model
        bidder.draw_valuations_()
        utility = learner.update_strategy_and_evaluate_utility()

        #writer.add_scalar('eval/utility', utility, e)
        if e % print_epoch == 0:
            print("Epoch {}:  \tutility: {:2f}".format(e, utility))

        if e % plot_epoch == 0:
            ##plot current function output
            bidder = strat_to_bidder(model, BATCH_SIZE)
            bidder.draw_valuations_()
            v = bidder.valuations
            b = bidder.get_action()
            share = b.mean()/v.mean()
            diff = (b-v).mean()
            writer.add_scalar('eval/share', share, e)
            writer.add_scalar('eval/diff', diff, e) 
            plot_bid_function(fig, v,b,writer,e)